## Step 1: Install Dependencies

In [1]:
# Install required packages using pip magic command (best for Jupyter)
%pip install -q gradio openai python-dotenv

print("✅ All packages installed!")

/Users/retlawair/Desktop/deploying-ai/deploying-ai-env/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
✅ All packages installed!


## Step 2: Load Environment Variables

In [2]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

import os
api_key = os.getenv('API_GATEWAY_KEY')
print(f"✅ API Key loaded: {api_key[:10] if api_key else 'NOT FOUND'}...")

✅ API Key loaded: YTLIZ27JW0...


## Step 3: Import Libraries

In [3]:
import gradio as gr
from openai import OpenAI
import os
from datetime import datetime

print(f"✅ Gradio {gr.__version__}")
print("✅ OpenAI SDK ready")

✅ Gradio 5.49.1
✅ OpenAI SDK ready


In [4]:
from pydantic import BaseModel, Field, field_validator
from typing import Optional, List, Literal
from enum import Enum

print("✅ Pydantic models ready")

✅ Pydantic models ready


## Step 4: Initialize OpenAI Client

In [5]:
# Initialize OpenAI client
client = OpenAI(
    default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')},
    base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1'
)

print("✅ OpenAI client initialized")

✅ OpenAI client initialized


In [6]:
# Define allowed values as enums for type safety
class ToneType(str, Enum):
    FORMAL = "formal"
    CASUAL = "casual"
    FRIENDLY = "friendly"
    PROFESSIONAL = "professional"
    HUMOROUS = "humorous"

class StyleType(str, Enum):
    CONCISE = "concise"
    DETAILED = "detailed"
    CREATIVE = "creative"
    ANALYTICAL = "analytical"
    SOCRATIC = "socratic"

class RoleType(str, Enum):
    ASSISTANT = "helpful AI assistant"
    MENTOR = "mentor and educator"
    EXPERT = "subject matter expert"
    BRAINSTORMER = "creative brainstorming partner"
    THERAPIST = "empathetic listener"
    ANALYST = "data analyst"

# Core configuration model
class ChatConfig(BaseModel):
    """Pydantic model for chat personality and behavior configuration"""
    
    tone: ToneType = Field(
        default=ToneType.FRIENDLY,
        description="The tone of voice for responses"
    )
    conversational_style: StyleType = Field(
        default=StyleType.CONCISE,
        description="How detailed or creative responses should be"
    )
    role: RoleType = Field(
        default=RoleType.ASSISTANT,
        description="The character role the AI adopts"
    )
    temperature: float = Field(
        default=0.7,
        ge=0.0,
        le=2.0,
        description="Controls randomness (0=deterministic, 2=very creative)"
    )
    max_tokens: int = Field(
        default=1000,
        ge=100,
        le=4000,
        description="Maximum response length"
    )
    custom_system_prompt: Optional[str] = Field(
        default=None,
        description="Override default system prompt"
    )
    model: str = Field(
        default="gpt-4o-mini",
        description="OpenAI model to use"
    )
    
    @field_validator('temperature')
    @classmethod
    def validate_temperature(cls, v):
        if v < 0 or v > 2:
            raise ValueError('Temperature must be between 0 and 2')
        return v
    
    def get_system_prompt(self) -> str:
        """Generate comprehensive system prompt based on configuration"""
        if self.custom_system_prompt:
            return self.custom_system_prompt
        
        return f"""You are a {self.role.value} with a {self.tone.value} tone.
Your conversational style is {self.conversational_style.value}.

Guidelines:
- Maintain consistency with your assigned tone throughout
- Adapt your response length based on your style (concise = short, detailed = thorough)
- Stay in character as a {self.role.value}
- Be helpful, accurate, and respectful"""
    
    class Config:
        use_enum_values = True

# Message model for type-safe message handling
class Message(BaseModel):
    role: Literal["system", "user", "assistant"]
    content: str

# Chat session model to track state
class ChatSession(BaseModel):
    config: ChatConfig
    messages: List[Message] = Field(default_factory=list)
    
    def add_message(self, role: str, content: str):
        """Add message to session"""
        self.messages.append(Message(role=role, content=content))
    
    def get_messages_for_api(self) -> List[dict]:
        """Convert messages to OpenAI API format"""
        return [{"role": msg.role, "content": msg.content} for msg in self.messages]

# Predefined personality configurations
class PersonalityLibrary:
    """Collection of pre-configured chat personalities"""
    
    DEFAULT = ChatConfig(
        tone=ToneType.FRIENDLY,
        conversational_style=StyleType.CONCISE,
        role=RoleType.ASSISTANT,
        temperature=0.7
    )
    
    MENTOR = ChatConfig(
        tone=ToneType.PROFESSIONAL,
        conversational_style=StyleType.DETAILED,
        role=RoleType.MENTOR,
        temperature=0.6
    )
    
    CREATIVE = ChatConfig(
        tone=ToneType.CASUAL,
        conversational_style=StyleType.CREATIVE,
        role=RoleType.BRAINSTORMER,
        temperature=0.9
    )
    
    ANALYTICAL = ChatConfig(
        tone=ToneType.FORMAL,
        conversational_style=StyleType.ANALYTICAL,
        role=RoleType.ANALYST,
        temperature=0.5
    )
    
    @staticmethod
    def get_all_personalities():
        """Return all available personalities"""
        return {
            "default": PersonalityLibrary.DEFAULT,
            "mentor": PersonalityLibrary.MENTOR,
            "creative": PersonalityLibrary.CREATIVE,
            "analytical": PersonalityLibrary.ANALYTICAL
        }

print("✅ Pydantic models and PersonalityLibrary defined")

✅ Pydantic models and PersonalityLibrary defined


## Step 4.5: Define Chat Configuration Models with Pydantic

## Step 5: Define Chat Function

In [7]:
def chat_with_gpt(message, history, config=None):
    """
    Enhanced chat with GPT-4 using Pydantic configuration management.
    Maintains conversation history with configurable personality.
    
    Args:
        message: Current user message (string)
        history: List of message dictionaries with 'role' and 'content' keys
        config: ChatConfig Pydantic model (uses DEFAULT if None)
    
    Returns:
        Assistant's response (string)
    """
    if config is None:
        config = PersonalityLibrary.DEFAULT
    
    try:
        # Validate and ensure config is ChatConfig instance
        if not isinstance(config, ChatConfig):
            config = ChatConfig(**config) if isinstance(config, dict) else PersonalityLibrary.DEFAULT
        
        # Build messages list with system prompt and history
        messages = [
            {
                "role": "system",
                "content": config.get_system_prompt()
            }
        ]
        
        # Add conversation history (already in correct format)
        if history:
            messages.extend(history)
        
        # Add current message
        messages.append({"role": "user", "content": message})
        
        # Call OpenAI API with validated config
        response = client.chat.completions.create(
            model=config.model,
            messages=messages,
            temperature=config.temperature,
            max_tokens=config.max_tokens
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Enhanced chat function defined with Pydantic config")

✅ Enhanced chat function defined with Pydantic config


## Step 6: Create Gradio Interface

In [8]:
# Store config choices
config_choices = {
    "personality": "default",
    "tone": "friendly", 
    "style": "concise",
    "temperature": 0.7
}

def simple_chat(message, history):
    """Simple chat function that reads from config_choices"""
    try:
        # Get base config from personality
        personalities = PersonalityLibrary.get_all_personalities()
        base_config = personalities.get(config_choices["personality"], PersonalityLibrary.DEFAULT)
        
        # Use selected tone and style directly as strings
        tone_str = config_choices["tone"]
        style_str = config_choices["style"]
        
        # Create simple system prompt with selected attributes
        system_prompt = f"""You are a helpful AI assistant with a {tone_str} tone.
Your conversational style is {style_str}.
Be helpful, accurate, and respectful."""
        
        # Build messages
        messages = [{"role": "system", "content": system_prompt}]
        if history:
            messages.extend(history)
        messages.append({"role": "user", "content": message})
        
        # Call API
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=config_choices["temperature"],
            max_tokens=1000
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Walter's AI Chatbot")
    gr.Markdown("Customize your chat experience with different personalities and settings")
    
    with gr.Row():
        personality = gr.Dropdown(
            choices=list(PersonalityLibrary.get_all_personalities().keys()),
            value="default",
            label="🎭 Personality"
        )
        tone = gr.Dropdown(
            choices=[t.value for t in ToneType],
            value="friendly",
            label="🎵 Tone"
        )
        style = gr.Dropdown(
            choices=[s.value for s in StyleType],
            value="concise",
            label="📝 Style"
        )
        temperature = gr.Slider(
            minimum=0.0,
            maximum=2.0,
            value=0.7,
            step=0.1,
            label="🔥 Temperature"
        )
    
    personality.change(lambda x: config_choices.update({"personality": x}), inputs=personality)
    tone.change(lambda x: config_choices.update({"tone": x}), inputs=tone)
    style.change(lambda x: config_choices.update({"style": x}), inputs=style)
    temperature.change(lambda x: config_choices.update({"temperature": x}), inputs=temperature)
    
    chatbot = gr.ChatInterface(
        fn=simple_chat,
        examples=[
            "What is machine learning?",
            "Explain quantum computing",
            "How do neural networks work?",
            "What is the capital of France?"
        ],
        type="messages"
    )

print("✅ Chatbot interface created")

✅ Chatbot interface created


## Step 7: Launch the Application

In [9]:
# Launch the Gradio app
print("\n" + "="*70)
print("🚀 LAUNCHING CHATBOT APPLICATION".center(70))
print("="*70)
print()
print("📱 Open your browser and navigate to: http://localhost:7860")
print()
print("✨ Features:")
print("   • Full conversation history")
print("   • Real-time responses from GPT-4")
print("   • Beautiful, responsive interface")
print("   • One-click chat management")
print()
print("⏹️  Press Ctrl+C in terminal to stop")
print()
print("="*70 + "\n")

# Launch
demo.launch(
    share=False,
    server_name="127.0.0.1",
    server_port=None,
    show_error=True,
    quiet=True
)


                   🚀 LAUNCHING CHATBOT APPLICATION                    

📱 Open your browser and navigate to: http://localhost:7860

✨ Features:
   • Full conversation history
   • Real-time responses from GPT-4
   • Beautiful, responsive interface
   • One-click chat management

⏹️  Press Ctrl+C in terminal to stop


